In [1]:
# import some common libraries
import numpy as np
import cv2
import random

# Helper Library
import os
import json
from PIL import Image
import matplotlib
from matplotlib import pyplot as plt

## Matching

In [2]:
def echo(string, padding=80):
    padding = " " * (padding - len(string)) if padding else ""
    print(string + padding, end='\r')

In [3]:
def cv2_imshow(img):
    %matplotlib inline
    img = img[:,:,[2,1,0]]
    img = Image.fromarray(img, mode="RGB")
    plt.figure(figsize=(20, 20))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    %matplotlib agg
    
def cv2_imshow_small(img):
    %matplotlib inline
    img = img[:,:,[2,1,0]]
    img = Image.fromarray(img, mode="RGB")
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    %matplotlib agg

In [4]:
def merge(source, destination):
    """
    run me with nosetests --with-doctest file.py

    >>> a = { 'first' : { 'all_rows' : { 'pass' : 'dog', 'number' : '1' } } }
    >>> b = { 'first' : { 'all_rows' : { 'fail' : 'cat', 'number' : '5' } } }
    >>> merge(b, a) == { 'first' : { 'all_rows' : { 'pass' : 'dog', 'fail' : 'cat', 'number' : '5' } } }
    True
    """
    for key, value in source.items():
        if isinstance(value, dict):
            # get node or create one
            node = destination.setdefault(key, {})
            merge(value, node)
        else:
            destination[key] = value

    return destination

In [203]:
# Check smaller width dimension of 2 images
# Smaller images: check width x dimension and choose the smaller
# Resize higher image resolution with the smaller dimension (width / height)
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

def get_downscaled_images(img1, img2):
    img1_h, img1_w, img1_c = img1.shape
    img2_h, img2_w, img2_c = img2.shape
    
    if img1_h > img2_h:
        scaled_img1 = image_resize(img1, width=img2_w)
        scaled_img2 = img2
    else:
        scaled_img2 = image_resize(img2, width=img1_w)
        scaled_img1 = img1
        
    
    return scaled_img1, scaled_img2

In [6]:
import json
def save_as_json(filename, folder, data):
    with open(folder+'/'+filename+'.json', 'w+') as fp:
        json.dump(data, fp)

#### ORB

In [67]:
# Get ORB Keypoints and Descriptor
n_features = 3000
factor = 2.0
orb = cv2.ORB_create(n_features, factor)

def get_orb_kp_and_des(img_path):
    img1 = cv2.imread(img_path)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    keypoints_img, descriptor_img =  orb.detectAndCompute(img1_gray, None)
    return keypoints_img, descriptor_img

def get_orb_kp_and_des_by_img(img1):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    keypoints_img, descriptor_img =  orb.detectAndCompute(img1_gray, None)
    return keypoints_img, descriptor_img

#### SIFT

In [8]:
# ORB matcher function
sift = cv2.SIFT_create()

def get_sift_kp_and_des(img_path):
    img1 = cv2.imread(img_path)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    keypoints_img, descriptor_img =  sift.detectAndCompute(img1_gray, None)
    return keypoints_img, descriptor_img

def get_sift_kp_and_des_by_img(img1):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    keypoints_img, descriptor_img =  sift.detectAndCompute(img1_gray, None)
    return keypoints_img, descriptor_img

#### Create a cropped image dictionary
```json
{
    "IMAGE_INDEX": {
        "selfies": {
            0: {
                "path": String,
                "format": String,
                "original_width": Number,
                "original_height": Number,
            },...
         },
        "cards": {
            0: {
                "path": String,
                "format": String,
                "original_width": Number,
                "original_height": Number,
            }, ...
        }
    }
}
```

In [211]:
# Create dict for evaluate image matching fature
# shape = h, w
def get_image_matching_dict(cropped_image_folder):
    images_dict = {}
    for filename in os.listdir(cropped_image_folder):
        filename_number = filename.split('_')[0]
        filename_type = filename.split('_')[1]
        
        file_index = filename.split('_')[len(filename.split('_')) - 1]
        file_index = int(file_index.split('.')[0])
        if filename_number not in images_dict:
            images_dict[filename_number] = {
                "selfies": None,
                "cards": None
            }
        
        
        if filename_type.find("selfie") != -1:
            # e-ktp image
            if images_dict[filename_number]["selfies"] is None:
                images_dict[filename_number]["selfies"] = {}
            
            
            images_dict[filename_number]["selfies"][file_index] = {
                'path': os.path.join(cropped_image_folder,filename),
                'format': filename.split('.')[1]
            }
        else:
            if images_dict[filename_number]["cards"] is None:
                images_dict[filename_number]["cards"] = {}

            images_dict[filename_number]["cards"][file_index] = {
                'path': os.path.join(cropped_image_folder,filename),
                'format': filename.split('.')[1]
            }
    
    return images_dict

matching_image_dict = get_image_matching_dict('./results/cropped/e-ktp')

#### Create a results image matching dictionary
```json
```

#### Remove underlier matches
*This function is still debateable, since currently author don't know what is the "good" ratio for scoring ORB or SIFT in this study*


In [213]:
# Set treshold to 0.7 based on Lowe paper ratio for SIFT
def get_good_sift_matches(matches):
    good = []
    for m,n in matches:
        if m.distance < 0.7 * n.distance:
            good.append(m)
            
    return good

# Set treshold to 64.0 based on ORB original paper ratio
def get_good_orb_matches(matches):
    good = []
    for m in matches:
        if m.distance < 64:
            good.append(m)
            
    return good

In [214]:
def get_matches_results_json(matching_image_dict, img_query_idx, img_train_idx, resize=False):
    matching_image_query = matching_image_dict[img_query_idx]
    matching_image_train = matching_image_dict[img_train_idx]
    sift_bf = cv2.BFMatcher()
    orb_bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    results = []
    for matching_image_type_query_key in matching_image_query:
        if img_query_idx == img_train_idx and matching_image_type_query_key == "cards":
            continue
        for matching_image_idx_query_key in matching_image_query[matching_image_type_query_key]:
            # Query selfie to train selfies and ORB
            if not resize:
                query_orb_keypoints, query_orb_descriptor = get_orb_kp_and_des(matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"])
                query_sift_keypoints, query_sift_descriptor = get_sift_kp_and_des(matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"])
            
                total_query_sift_keypoints = len(query_sift_keypoints)
                total_query_orb_keypoints = len(query_orb_keypoints)
                query_image = cv2.imread(matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"])

            if not(img_query_idx == img_train_idx and "selfies" == matching_image_type_query_key):  # ignoring the same image
                for matching_image_selfie_train_key in matching_image_train["selfies"]:
                    train_image = cv2.imread(matching_image_train["selfies"][matching_image_selfie_train_key]["path"])
                    if resize:
                        query_image = cv2.imread(matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"])
                        train_image, query_image = get_downscaled_images(train_image, query_image)
                        
                        query_orb_keypoints, query_orb_descriptor = get_orb_kp_and_des_by_img(query_image)
                        query_sift_keypoints, query_sift_descriptor = get_sift_kp_and_des_by_img(query_image)
                             
                        train_orb_keypoints, train_orb_descriptor = get_orb_kp_and_des_by_img(train_image)
                        train_sift_keypoints, train_sift_descriptor = get_sift_kp_and_des_by_img(train_image)
                    else:
                        train_orb_keypoints, train_orb_descriptor = get_orb_kp_and_des(matching_image_train["selfies"][matching_image_selfie_train_key]["path"])
                        train_sift_keypoints, train_sift_descriptor = get_sift_kp_and_des(matching_image_train["selfies"][matching_image_selfie_train_key]["path"])
                    
                    total_train_sift_keypoints = len(train_sift_keypoints)
                    total_train_orb_keypoints = len(train_orb_keypoints)
                    
                    orb_matches = orb_bf.match(query_orb_descriptor,train_orb_descriptor)
                    results.append({
                        "query_image_index": img_query_idx,
                        "query_image_type": matching_image_type_query_key,
                        "query_image_type_index": matching_image_idx_query_key,
                        "query_image_path": matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"],
                        "query_total_keypoints": len(query_orb_keypoints),
                        "query_image_width": query_image.shape[1],
                        "query_image_height": query_image.shape[0],
                        "train_image_index": img_train_idx,
                        "train_image_type": "selfies",
                        "train_image_type_index": matching_image_selfie_train_key,
                        "train_image_path": matching_image_train["selfies"][matching_image_selfie_train_key]["path"],
                        "train_total_keypoints": total_train_orb_keypoints,
                        "train_image_width": train_image.shape[1],
                        "train_image_height": train_image.shape[0],
                        "matching_method": "ORB",
                        "total_matches": len(orb_matches),
                        "total_good_matches": len(get_good_orb_matches(orb_matches)),
                        "with_resize": resize 
                    })
                    
                    sift_matches = sift_bf.knnMatch(query_sift_descriptor, train_sift_descriptor, k=2)
                    results.append({
                        "query_image_index": img_query_idx,
                        "query_image_type": matching_image_type_query_key,
                        "query_image_type_index": matching_image_idx_query_key,
                        "query_image_path": matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"],
                        "query_total_keypoints": len(query_sift_keypoints),
                        "query_image_width": query_image.shape[1],
                        "query_image_height": query_image.shape[0],
                        "train_image_index": img_train_idx,
                        "train_image_type": "selfies",
                        "train_image_type_index": matching_image_selfie_train_key,
                        "train_image_path": matching_image_train["selfies"][matching_image_selfie_train_key]["path"],
                        "train_total_keypoints": total_train_sift_keypoints,
                        "train_image_width": train_image.shape[1],
                        "train_image_height": train_image.shape[0],
                        "matching_method": "SIFT",
                        "total_matches": len(sift_matches),
                        "total_good_matches": len(get_good_sift_matches(sift_matches)),
                        "with_resize": resize 
                    })

            # Query selfie to train cards and ORB
            if not(img_query_idx == img_train_idx and "cards" == matching_image_type_query_key): # ignoring the same image
                for matching_image_card_train_key in matching_image_train["cards"]:
                    train_image = cv2.imread(matching_image_train["cards"][matching_image_card_train_key]["path"])
                    if resize:
                        query_image = cv2.imread(matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"])
                        train_image, query_image = get_downscaled_images(train_image, query_image)
                        
                        query_orb_keypoints, query_orb_descriptor = get_orb_kp_and_des_by_img(query_image)
                        query_sift_keypoints, query_sift_descriptor = get_sift_kp_and_des_by_img(query_image)
                             
                        train_orb_keypoints, train_orb_descriptor = get_orb_kp_and_des_by_img(train_image)
                        train_sift_keypoints, train_sift_descriptor = get_sift_kp_and_des_by_img(train_image)
                    else:
                        train_orb_keypoints, train_orb_descriptor = get_orb_kp_and_des(matching_image_train["cards"][matching_image_card_train_key]["path"])
                        train_sift_keypoints, train_sift_descriptor = get_sift_kp_and_des(matching_image_train["cards"][matching_image_card_train_key]["path"])
                    
                    total_train_sift_keypoints = len(train_sift_keypoints)
                    total_train_orb_keypoints = len(train_orb_keypoints)
                    
                    orb_matches = orb_bf.match(query_orb_descriptor,train_orb_descriptor)
                    results.append({
                        "query_image_index": img_query_idx,
                        "query_image_type": matching_image_type_query_key,
                        "query_image_type_index": matching_image_idx_query_key,
                        "query_image_path": matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"],
                        "query_total_keypoints": len(query_orb_keypoints),
                        "query_image_width": query_image.shape[1],
                        "query_image_height": query_image.shape[0],
                        "train_image_index": img_train_idx,
                        "train_image_type": "cards",
                        "train_image_type_index": matching_image_card_train_key,
                        "train_image_path": matching_image_train["cards"][matching_image_card_train_key]["path"],
                        "train_total_keypoints": total_train_orb_keypoints,
                        "train_image_width": train_image.shape[1],
                        "train_image_height": train_image.shape[0],
                        "matching_method": "ORB",
                        "total_matches": len(orb_matches),
                        "total_good_matches": len(get_good_orb_matches(orb_matches)),
                        "with_resize": resize 
                    })
                    
                    sift_matches = sift_bf.knnMatch(query_sift_descriptor, train_sift_descriptor, k=2)
                    results.append({
                        "query_image_index": img_query_idx,
                        "query_image_type": matching_image_type_query_key,
                        "query_image_type_index": matching_image_idx_query_key,
                        "query_image_path": matching_image_query[matching_image_type_query_key][matching_image_idx_query_key]["path"],
                        "query_total_keypoints": len(query_sift_keypoints),
                        "query_image_width": query_image.shape[1],
                        "query_image_height": query_image.shape[0],
                        "train_image_index": img_train_idx,
                        "train_image_type": "cards",
                        "train_image_type_index": matching_image_card_train_key,
                        "train_image_path": matching_image_train["cards"][matching_image_card_train_key]["path"],
                        "train_total_keypoints": total_train_sift_keypoints,
                        "train_image_width": train_image.shape[1],
                        "train_image_height": train_image.shape[0],
                        "matching_method": "SIFT",
                        "total_matches": len(sift_matches),
                        "total_good_matches": len(get_good_sift_matches(sift_matches)),
                        "with_resize": resize
                    })
                
    return results   

In [22]:
# Reset matching folder files use it to reset the results
def delete_all_file_in_folder(path):
    # Delete previous cropped image
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [12]:
json_scenario_1_path = "./results/matching/scenario_1"
delete_all_file_in_folder(json_scenario_1_path)
print("All file in scenario 1 path deleted.")

All file in scenario 1 path deleted.


In [23]:
json_scenario_2_path = "./results/matching/scenario_2"
delete_all_file_in_folder(json_scenario_2_path)
print("All file in scenario 2 path deleted.")

All file in scenario 2 path deleted.


In [24]:
import pandas as pd

def save_all_scenario_json_into_csv(scenario_folder, csv_name, target_folder):
    all_df = pd.DataFrame([])
    for filename in os.listdir(scenario_folder):
        file_path = os.path.join(scenario_folder, filename)
        with open(file_path) as json_file:
            json_dict = json.load(json_file)
            df = pd.DataFrame.from_dict(pd.json_normalize(json_dict), orient='columns')
            all_df = pd.concat([all_df, df], ignore_index=True)
            
    all_df.to_csv(target_folder+csv_name+".csv")

### Scenario 1
Basically, we match each pair of dataset image (selfie image to card image) and see the results of the each dataset. And see whether there's some images that not match to its pair

In [216]:
scenario_1_results = {}
total_scenario_1 = len(matching_image_dict)
curr = 1
echo("Creating scenario 1 json")

import time
start_time = time.time()

for matching_image_key in matching_image_dict:
    results = get_matches_results_json(matching_image_dict, matching_image_key, matching_image_key)
    results_resized = get_matches_results_json(matching_image_dict, matching_image_key, matching_image_key, resize=True)
    results = results_resized + results
    save_as_json(matching_image_key+"_to_"+matching_image_key, "./results/matching/scenario_1", results)
    echo(str(curr)+" out of "+str(total_scenario_1)+" json has saved. Currently on "+matching_image_key+" to "+matching_image_key+". Execution time: %s seconds" % (time.time() - start_time))
    curr = curr + 1

print("Saving to directory scenario_1 completed!")

Saving to directory scenario_1 completed!on 99 to 99. Execution time: 144.16414308547974 secondss


TODO
---
- plotting some result of matching
    - scenario 1
    - scenario 2

- average good match on each method (sift and orb)
- lowest match on same pair of dataset on each method (sift and orb)
- lowest match on different pair of  on each method (sift and orb)dataset
- highest match on same pair of dataset on each method (sift and orb)
- lowest match on different pair of dataset on each method (sift and orb)
-  ...


- Check standard deviation of the data
    - For an approximate answer, please estimate your coefficient of variation (CV=standard deviation / mean).As a rule of thumb, a CV >= 1 indicates a relatively high variation, while a CV < 1 can be considered low. This means that distributions with a coefficient of variation higher than 1 are considered to be high variance whereas those with a CV lower than 1 are considered to be low-variance. 
    - Remember,  standard deviations aren't "good" or "bad". They are indicators of how spread out your data is. A "good" SD depends if you expect your distribution to be centered or spread out around the mean. This really depends on your data.

In [217]:
json_scenario_1_path = "./results/matching/scenario_1"
save_all_scenario_json_into_csv(json_scenario_1_path, "scenario_1", "./results/matching/")
print("Saving into csv completed !")

Saving into csv completed !


#### Evaluation Scenario 1
Plotting all the results into charts, so we can analyze what is the results of this study

In [218]:
scenario_1_df = pd.read_csv("./results/matching/scenario_1.csv")

In [219]:
scenario_1_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000
mean,277.500000,73.510791,0.050360,1233.836331,743.705036,489.359712,73.510791,0.050360,3979.116906,1132.104317,748.402878,868.120504,309.971223
std,160.647648,38.123435,0.218883,1417.911335,432.775063,288.634699,38.123435,0.249648,6860.729355,885.733171,575.974183,1360.522380,274.040812
min,0.000000,7.000000,0.000000,1.000000,103.000000,92.000000,7.000000,0.000000,3.000000,182.000000,98.000000,1.000000,0.000000
25%,138.750000,39.000000,0.000000,427.000000,423.000000,282.000000,39.000000,0.000000,1239.000000,501.000000,342.000000,291.000000,106.750000
50%,277.500000,75.000000,0.000000,839.000000,611.000000,376.000000,75.000000,0.000000,2307.000000,751.500000,514.500000,562.500000,245.000000
75%,416.250000,108.000000,0.000000,1740.750000,998.000000,662.000000,108.000000,0.000000,2908.000000,1497.000000,973.000000,940.500000,435.250000
max,555.000000,138.000000,1.000000,13898.000000,2971.000000,1977.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,13898.000000,2134.000000


In [220]:
scenario_1_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
0,0,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1089,562,374,ORB,540,515,True
1,1,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1138,562,374,SIFT,1281,468,True
2,2,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1764,719,478,ORB,348,317,False
3,3,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1897,719,478,SIFT,1281,409,False
4,4,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1950,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,2823,1562,982,ORB,793,764,True


Evaluation SIFT Method

In [221]:
scenario_1_sift_df = scenario_1_df[(scenario_1_df.matching_method == "SIFT")]

In [222]:
scenario_1_sift_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,278.000000,278.000000,278.00000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000
mean,278.000000,73.510791,0.05036,1311.399281,743.705036,489.359712,73.510791,0.050360,5936.823741,1132.104317,748.402878,1311.399281,230.866906
std,160.791791,38.157827,0.21908,1796.459274,433.165478,288.895082,38.157827,0.249873,9271.292732,886.532209,576.493781,1796.459274,229.343200
min,1.000000,7.000000,0.00000,6.000000,103.000000,92.000000,7.000000,0.000000,25.000000,182.000000,98.000000,6.000000,0.000000
25%,139.500000,39.250000,0.00000,467.250000,423.000000,282.000000,39.250000,0.000000,1065.000000,506.000000,342.500000,467.250000,87.000000
50%,278.000000,75.000000,0.00000,805.000000,611.000000,376.000000,75.000000,0.000000,2284.000000,751.500000,514.500000,805.000000,172.000000
75%,416.500000,107.750000,0.00000,1354.000000,991.750000,656.750000,107.750000,0.000000,5942.000000,1496.250000,968.000000,1354.000000,332.250000
max,555.000000,138.000000,1.00000,13898.000000,2971.000000,1977.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,13898.000000,2134.000000


In [223]:
scenario_1_sift_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
1,1,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1138,562,374,SIFT,1281,468,True
3,3,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1897,719,478,SIFT,1281,409,False
5,5,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1861,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,22688,1562,982,SIFT,1861,403,True
7,7,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1861,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,42602,2741,1723,SIFT,1861,399,False
9,9,102,selfies,0,./results/cropped/e-ktp\102_selfie_crop_0.jpg,3581,1461,916,102,cards,0,./results/cropped/e-ktp\102_card_crop_0.jpg,13246,1413,916,SIFT,3581,511,True


In [227]:
# Downscaled dataframe
scenario_1_sift_with_downscale_df = scenario_1_sift_df[scenario_1_sift_df.with_resize == True]
scenario_1_sift_with_downscale_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
1,1,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1138,562,374,SIFT,1281,468,True
5,5,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1861,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,22688,1562,982,SIFT,1861,403,True
9,9,102,selfies,0,./results/cropped/e-ktp\102_selfie_crop_0.jpg,3581,1461,916,102,cards,0,./results/cropped/e-ktp\102_card_crop_0.jpg,13246,1413,916,SIFT,3581,511,True
13,13,103,selfies,0,./results/cropped/e-ktp\103_selfie_crop_0.jpeg,1356,688,457,103,cards,0,./results/cropped/e-ktp\103_card_crop_0.jpeg,2093,709,457,SIFT,1356,483,True
17,17,104,selfies,0,./results/cropped/e-ktp\104_selfie_crop_0.jpeg,866,456,289,104,cards,0,./results/cropped/e-ktp\104_card_crop_0.jpeg,930,424,289,SIFT,866,264,True


In [228]:
scenario_1_sift_with_downscale_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000
mean,276.769784,73.510791,0.050360,1237.956835,728.417266,479.431655,73.510791,0.050360,3339.496403,725.618705,479.431655,1237.956835,235.244604
std,161.023552,38.226891,0.219477,1682.552706,401.189714,266.535639,38.226891,0.250326,4716.274324,416.147486,266.535639,1682.552706,226.994664
min,1.000000,7.000000,0.000000,6.000000,225.000000,225.000000,7.000000,0.000000,25.000000,254.000000,225.000000,6.000000,0.000000
25%,139.000000,39.500000,0.000000,501.000000,413.000000,275.000000,39.500000,0.000000,775.500000,418.500000,275.000000,501.000000,91.000000
50%,277.000000,75.000000,0.000000,781.000000,599.000000,369.000000,75.000000,0.000000,1281.000000,538.000000,369.000000,781.000000,174.000000
75%,414.000000,107.500000,0.000000,1310.500000,966.000000,621.500000,107.500000,0.000000,3518.500000,977.000000,621.500000,1310.500000,340.000000
max,553.000000,138.000000,1.000000,13898.000000,1950.000000,1206.000000,138.000000,2.000000,22688.000000,1895.000000,1206.000000,13898.000000,2039.000000


Evaluation ORB Method

In [229]:
scenario_1_orb_df = scenario_1_df[(scenario_1_df.matching_method == "ORB")]

In [230]:
scenario_1_orb_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,278.000000,278.000000,278.00000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.00000
mean,277.000000,73.510791,0.05036,1156.273381,743.705036,489.359712,73.510791,0.050360,2021.410072,1132.104317,748.402878,424.841727,389.07554
std,160.791791,38.157827,0.21908,888.178896,433.165478,288.895082,38.157827,0.249873,812.014536,886.532209,576.493781,295.038754,292.07769
min,0.000000,7.000000,0.00000,1.000000,103.000000,92.000000,7.000000,0.000000,3.000000,182.000000,98.000000,1.000000,0.00000
25%,138.500000,39.250000,0.00000,360.250000,423.000000,282.000000,39.250000,0.000000,1435.250000,506.000000,342.500000,170.250000,146.00000
50%,277.000000,75.000000,0.00000,940.000000,611.000000,376.000000,75.000000,0.000000,2327.500000,751.500000,514.500000,373.000000,323.00000
75%,415.500000,107.750000,0.00000,1953.750000,991.750000,656.750000,107.750000,0.000000,2701.500000,1496.250000,968.000000,611.000000,577.00000
max,554.000000,138.000000,1.00000,2917.000000,2971.000000,1977.000000,138.000000,2.000000,2964.000000,4471.000000,2980.000000,1319.000000,1285.00000


In [231]:
scenario_1_orb_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
0,0,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1089,562,374,ORB,540,515,True
2,2,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1764,719,478,ORB,348,317,False
4,4,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1950,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,2823,1562,982,ORB,793,764,True
6,6,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1950,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,2917,2741,1723,ORB,690,636,False
8,8,102,selfies,0,./results/cropped/e-ktp\102_selfie_crop_0.jpg,2780,1461,916,102,cards,0,./results/cropped/e-ktp\102_card_crop_0.jpg,2823,1413,916,ORB,822,759,True


In [232]:
col = scenario_1_orb_df[(scenario_1_orb_df.total_good_matches < 900)]
col.count()
# < 100: 57
# < 200: 34
# < 300: 40
# < 400: 35
# < 500: 18
# < 600: 26
# < 700: 13
# < 800: 17
# < 900: 15


Unnamed: 0                258
query_image_index         258
query_image_type          258
query_image_type_index    258
query_image_path          258
query_total_keypoints     258
query_image_width         258
query_image_height        258
train_image_index         258
train_image_type          258
train_image_type_index    258
train_image_path          258
train_total_keypoints     258
train_image_width         258
train_image_height        258
matching_method           258
total_matches             258
total_good_matches        258
with_resize               258
dtype: int64

In [233]:
# Downscaled dataframe
scenario_1_orb_with_downscale_df = scenario_1_orb_df[scenario_1_orb_df.with_resize == True]
scenario_1_orb_with_downscale_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
0,0,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1089,562,374,ORB,540,515,True
4,4,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1950,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,2823,1562,982,ORB,793,764,True
8,8,102,selfies,0,./results/cropped/e-ktp\102_selfie_crop_0.jpg,2780,1461,916,102,cards,0,./results/cropped/e-ktp\102_card_crop_0.jpg,2823,1413,916,ORB,822,759,True
12,12,103,selfies,0,./results/cropped/e-ktp\103_selfie_crop_0.jpeg,1568,688,457,103,cards,0,./results/cropped/e-ktp\103_card_crop_0.jpeg,2246,709,457,ORB,980,966,True
16,16,104,selfies,0,./results/cropped/e-ktp\104_selfie_crop_0.jpeg,921,456,289,104,cards,0,./results/cropped/e-ktp\104_card_crop_0.jpeg,1466,424,289,ORB,480,469,True


In [234]:
scenario_1_orb_with_downscale_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000
mean,275.769784,73.510791,0.050360,1136.014388,728.417266,479.431655,73.510791,0.050360,1733.258993,725.618705,479.431655,489.345324,463.107914
std,161.023552,38.226891,0.219477,861.236708,401.189714,266.535639,38.226891,0.250326,823.235006,416.147486,266.535639,334.136671,332.398785
min,0.000000,7.000000,0.000000,1.000000,225.000000,225.000000,7.000000,0.000000,4.000000,254.000000,225.000000,1.000000,1.000000
25%,138.000000,39.500000,0.000000,396.500000,413.000000,275.000000,39.500000,0.000000,1061.000000,418.500000,275.000000,206.000000,167.500000
50%,276.000000,75.000000,0.000000,921.000000,599.000000,369.000000,75.000000,0.000000,1797.000000,538.000000,369.000000,436.000000,413.000000
75%,413.000000,107.500000,0.000000,1891.000000,966.000000,621.500000,107.500000,0.000000,2512.500000,977.000000,621.500000,793.500000,759.000000
max,552.000000,138.000000,1.000000,2837.000000,1950.000000,1206.000000,138.000000,2.000000,2835.000000,1895.000000,1206.000000,1319.000000,1285.000000


In [235]:
# Downscaled dataframe
scenario_1_orb_without_downscale_df = scenario_1_orb_df[scenario_1_orb_df.with_resize == False]
scenario_1_orb_without_downscale_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
2,2,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1764,719,478,ORB,348,317,False
6,6,101,selfies,0,./results/cropped/e-ktp\101_selfie_crop_0.jpg,1950,1521,982,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,2917,2741,1723,ORB,690,636,False
10,10,102,selfies,0,./results/cropped/e-ktp\102_selfie_crop_0.jpg,2780,1461,916,102,cards,0,./results/cropped/e-ktp\102_card_crop_0.jpg,2875,2129,1380,ORB,709,601,False
14,14,103,selfies,0,./results/cropped/e-ktp\103_selfie_crop_0.jpeg,1751,707,469,103,cards,0,./results/cropped/e-ktp\103_card_crop_0.jpeg,2246,709,457,ORB,1008,991,False
18,18,104,selfies,0,./results/cropped/e-ktp\104_selfie_crop_0.jpeg,921,456,289,104,cards,0,./results/cropped/e-ktp\104_card_crop_0.jpeg,2391,751,511,ORB,402,375,False


In [236]:
scenario_1_orb_without_downscale_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000
mean,278.230216,73.510791,0.050360,1176.532374,758.992806,499.287770,73.510791,0.050360,2309.561151,1538.589928,1017.374101,360.338129,315.043165
std,161.132600,38.226891,0.219477,916.997505,463.896876,310.299394,38.226891,0.250326,691.760023,1035.151895,671.057140,233.874906,222.950598
min,2.000000,7.000000,0.000000,1.000000,103.000000,92.000000,7.000000,0.000000,3.000000,182.000000,98.000000,1.000000,0.000000
25%,141.000000,39.500000,0.000000,327.000000,440.500000,286.000000,39.500000,0.000000,2139.500000,711.000000,474.500000,144.500000,117.500000
50%,278.000000,75.000000,0.000000,946.000000,640.000000,382.000000,75.000000,0.000000,2408.000000,971.000000,642.000000,343.000000,278.000000
75%,417.000000,107.500000,0.000000,1981.500000,1028.000000,685.500000,107.500000,0.000000,2847.000000,2466.500000,1606.500000,557.000000,502.000000
max,554.000000,138.000000,1.000000,2917.000000,2971.000000,1977.000000,138.000000,2.000000,2964.000000,4471.000000,2980.000000,1008.000000,991.000000


### Scenario 2
In this scenario, we match each of dataset to another dataset (1 to many). It should take a lot of time to match since we will basically do all the combination.

If we have 10 dataset, then total process is **1 * 9 * 10 = 90 tests**

We save the data for helping plotting the data easily

In [241]:
matching_done = {}

In [244]:
# your code goes here
total_scenario_2 = len(matching_image_dict) * (len(matching_image_dict) - 1)
curr = 1
echo("Creating scenario 2 json")

import time
start_time = time.time()

for matching_image_query_key in matching_image_dict:
    current_image_query_result = {}
    for matching_image_train_key in matching_image_dict:
        if matching_image_train_key == matching_image_query_key or (matching_image_query_key in matching_done and matching_image_train_key in matching_done[matching_image_query_key]):
            curr = curr + 1
            continue
            
        # For debugging purposes
        file_exists_1 = os.path.isfile('./results/matching/scenario_2/'+matching_image_query_key+"_to_"+matching_image_train_key+".json")
        file_exists_2 = os.path.isfile('./results/matching/scenario_2/'+matching_image_train_key+"_to_"+matching_image_query_key+".json")
        if file_exists_1 or file_exists_2:
            if matching_image_train_key not in matching_done:
                matching_done[matching_image_train_key] = {}
            matching_done[matching_image_train_key][matching_image_query_key] = 1
            curr = curr + 1
            continue
            
        if matching_image_train_key not in matching_done:
            matching_done[matching_image_train_key] = {}
        matching_done[matching_image_train_key][matching_image_query_key] = 1
        
        echo(str(curr)+" out of "+str(total_scenario_2)+" json has saved. Currently on "+matching_image_query_key+" to "+matching_image_train_key+". Execution time: %s seconds" % (time.time() - start_time))
        
        results = get_matches_results_json(matching_image_dict, matching_image_query_key, matching_image_train_key) # , with_cuda=True
        results_resized = get_matches_results_json(matching_image_dict, matching_image_query_key, matching_image_train_key, resize=True) #, with_cuda=True
        results = results_resized + results
        save_as_json(matching_image_query_key+"_to_"+matching_image_train_key, "./results/matching/scenario_2", results)
        curr = curr + 1

            
print("Saving to directory scenario_2 completed!")

Saving to directory scenario_2 completed!tly on 98 to 99. Execution time: 6235.857724905014 secondss


In [245]:
json_scenario_2_path = "./results/matching/scenario_2"
save_all_scenario_json_into_csv(json_scenario_2_path, "scenario_2", "./results/matching/")
print("Saving into csv completed !")

Saving into csv completed !


#### Evaluation Scenario 2
Plotting all the results into charts, so we can analyze what is the results of this study

In [246]:
# your evaluation code goes here
scenario_2_df = pd.read_csv("./results/matching/scenario_2.csv")

In [247]:
scenario_2_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000,139316.000000
mean,69657.500000,79.217979,0.046599,2647.153708,891.961368,591.913563,67.276436,0.043527,2510.266437,957.705920,631.401016,2025.902036,211.851288
std,40217.209389,43.082960,0.226790,5362.452842,720.061126,471.723774,31.747944,0.222743,5036.663323,757.640114,496.441066,5476.884147,288.303632
min,0.000000,7.000000,0.000000,0.000000,103.000000,92.000000,7.000000,0.000000,1.000000,103.000000,92.000000,0.000000,0.000000
25%,34828.750000,36.000000,0.000000,688.000000,456.000000,303.000000,43.000000,0.000000,636.000000,467.000000,304.000000,276.000000,38.000000
50%,69657.500000,101.000000,0.000000,1640.000000,686.000000,446.000000,68.000000,0.000000,1458.000000,701.000000,468.000000,592.000000,112.000000
75%,104486.250000,117.000000,0.000000,2491.000000,971.000000,642.000000,90.000000,0.000000,2447.000000,1102.000000,709.000000,1390.000000,346.000000
max,139315.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,55958.000000,43499.000000


In [266]:
scenario_2_df_ = scenario_2_df[(scenario_2_df.query_image_index != 133) & (scenario_2_df.train_image_index != 134)]

In [267]:
scenario_2_df_.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000,137212.000000
mean,69802.762280,78.512346,0.047022,2575.969237,886.401612,588.424773,67.068274,0.043786,2478.519073,954.018774,629.043626,1958.510036,209.037832
std,40439.457629,42.952900,0.227869,5119.826210,719.688076,471.641908,31.495783,0.223287,4938.720600,757.334223,496.280801,5231.115630,232.729103
min,0.000000,7.000000,0.000000,0.000000,103.000000,92.000000,7.000000,0.000000,1.000000,103.000000,92.000000,0.000000,0.000000
25%,34582.750000,35.000000,0.000000,671.000000,456.000000,303.000000,43.000000,0.000000,628.750000,463.000000,303.000000,272.000000,38.000000
50%,69221.500000,100.000000,0.000000,1619.000000,678.000000,446.000000,68.000000,0.000000,1453.000000,701.000000,468.000000,589.000000,111.000000
75%,105012.250000,116.000000,0.000000,2491.000000,960.000000,633.000000,90.000000,0.000000,2440.000000,1102.000000,699.000000,1376.000000,342.000000
max,139315.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,55958.000000,8053.000000


In [268]:
col = scenario_2_df_[scenario_2_df_.total_good_matches >= 8053]
col

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
136169,136169,7,cards,0,./results/cropped/e-ktp\7_card_crop_0.jpg,55958,2585,1690,83,selfies,0,./results/cropped/e-ktp\83_selfie_crop_0.jpeg,818,443,298,SIFT,55958,8053,NaN


In [248]:
scenario_2_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
0,0,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,10,selfies,0,./results/cropped/e-ktp\10_selfie_crop_0.jpg,1260,571,374,ORB,379,354,NaN
1,1,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,10,selfies,0,./results/cropped/e-ktp\10_selfie_crop_0.jpg,842,571,374,SIFT,1281,36,NaN
2,2,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1215,599,374,10,cards,0,./results/cropped/e-ktp\10_card_crop_0.jpg,2187,541,374,ORB,495,459,True
3,3,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,10,cards,0,./results/cropped/e-ktp\10_card_crop_0.jpg,2136,541,374,SIFT,1281,108,True
4,4,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1764,719,478,10,selfies,0,./results/cropped/e-ktp\10_selfie_crop_0.jpg,1826,730,478,ORB,486,450,NaN


In [ ]:
# Create new column percentage matches and keypoint

##### SIFT Scenario 2

In [249]:
scenario_2_sift_df = scenario_2_df[(scenario_2_df.matching_method == "SIFT")]

In [250]:
scenario_2_sift_df.tail()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
139307,139307,98,cards,0,./results/cropped/e-ktp\98_card_crop_0.jpeg,2254,671,471,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,1453,690,471,SIFT,2254,369,True
139309,139309,98,selfies,0,./results/cropped/e-ktp\98_selfie_crop_0.jpeg,501,421,251,99,selfies,0,./results/cropped/e-ktp\99_selfie_crop_0.jpeg,388,550,342,SIFT,501,109,NaN
139311,139311,98,selfies,0,./results/cropped/e-ktp\98_selfie_crop_0.jpeg,501,421,251,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,1453,690,471,SIFT,501,59,False
139313,139313,98,cards,0,./results/cropped/e-ktp\98_card_crop_0.jpeg,3037,763,535,99,selfies,0,./results/cropped/e-ktp\99_selfie_crop_0.jpeg,388,550,342,SIFT,3037,75,NaN
139315,139315,98,cards,0,./results/cropped/e-ktp\98_card_crop_0.jpeg,3037,763,535,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,1453,690,471,SIFT,3037,369,False


In [251]:
scenario_2_sift_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000,69658.000000
mean,69658.000000,79.217979,0.046599,3682.666528,891.961368,591.913563,67.276436,0.043527,3447.110325,957.705920,631.401016,3682.666528,97.954779
std,40217.353725,43.083114,0.226791,7378.694987,720.063710,471.725467,31.748058,0.222744,6932.627375,757.642833,496.442848,7378.694987,295.135373
min,1.000000,7.000000,0.000000,6.000000,103.000000,92.000000,7.000000,0.000000,6.000000,103.000000,92.000000,6.000000,0.000000
25%,34829.500000,36.000000,0.000000,651.000000,456.000000,303.000000,43.000000,0.000000,631.000000,467.000000,304.000000,651.000000,23.000000
50%,69658.000000,101.000000,0.000000,1389.000000,686.000000,446.000000,68.000000,0.000000,1283.000000,701.000000,468.000000,1389.000000,54.000000
75%,104486.500000,117.000000,0.000000,2865.000000,971.000000,642.000000,90.000000,0.000000,2669.000000,1102.000000,709.000000,2865.000000,113.000000
max,139315.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,55958.000000,43499.000000


In [252]:
# Downscaled dataframe
scenario_2_sift_with_downscale_df = scenario_2_sift_df[scenario_2_sift_df.with_resize == True]
scenario_2_sift_with_downscale_df.tail()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
139275,139275,97,cards,0,./results/cropped/e-ktp\97_card_crop_0.jpeg,1493,592,407,98,cards,0,./results/cropped/e-ktp\98_card_crop_0.jpeg,1502,580,407,SIFT,1493,234,True
139287,139287,97,selfies,0,./results/cropped/e-ktp\97_selfie_crop_0.jpeg,600,354,225,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,263,329,225,SIFT,600,48,True
139291,139291,97,cards,0,./results/cropped/e-ktp\97_card_crop_0.jpeg,1493,592,407,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,1094,596,407,SIFT,1493,163,True
139303,139303,98,selfies,0,./results/cropped/e-ktp\98_selfie_crop_0.jpeg,501,421,251,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,371,367,251,SIFT,501,87,True
139307,139307,98,cards,0,./results/cropped/e-ktp\98_card_crop_0.jpeg,2254,671,471,99,cards,0,./results/cropped/e-ktp\99_card_crop_0.jpeg,1453,690,471,SIFT,2254,369,True


In [253]:
scenario_2_sift_with_downscale_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000
mean,69681.047469,79.099811,0.046437,3111.362380,816.968526,542.761222,66.800092,0.049074,3330.573812,822.212578,542.750330,3111.362380,114.450553
std,40094.576440,43.169005,0.226446,5821.872357,542.961086,354.213035,31.496304,0.250214,5214.598186,543.025965,354.222479,5821.872357,358.745489
min,3.000000,7.000000,0.000000,6.000000,225.000000,225.000000,7.000000,0.000000,25.000000,257.000000,225.000000,6.000000,0.000000
25%,35025.000000,36.000000,0.000000,705.000000,485.000000,322.000000,42.000000,0.000000,1031.000000,492.000000,322.000000,705.000000,29.000000
50%,69843.000000,101.000000,0.000000,1528.000000,688.000000,457.000000,68.000000,0.000000,1924.000000,689.000000,457.000000,1528.000000,77.000000
75%,104277.000000,117.000000,0.000000,2798.000000,868.000000,576.000000,89.000000,0.000000,3229.000000,855.000000,576.000000,2798.000000,149.000000
max,139307.000000,138.000000,2.000000,55958.000000,4187.000000,2791.000000,138.000000,2.000000,55958.000000,4160.000000,2791.000000,55958.000000,43499.000000


In [254]:
# Without downscaled dataframe
scenario_2_sift_without_downscale_df = scenario_2_sift_df[scenario_2_sift_df.with_resize == False]
scenario_2_sift_without_downscale_df.head()

,Unnamed: 0,query_image_index,query_image_type,query_image_type_index,query_image_path,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type,train_image_type_index,train_image_path,train_total_keypoints,train_image_width,train_image_height,matching_method,total_matches,total_good_matches,with_resize
11,11,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,10,cards,0,./results/cropped/e-ktp\10_card_crop_0.jpg,38557,3457,2386,SIFT,1281,80,False
15,15,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1897,719,478,10,cards,0,./results/cropped/e-ktp\10_card_crop_0.jpg,38557,3457,2386,SIFT,1897,97,False
27,27,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,42602,2741,1723,SIFT,1281,81,False
31,31,100,cards,0,./results/cropped/e-ktp\100_card_crop_0.jpeg,1897,719,478,101,cards,0,./results/cropped/e-ktp\101_card_crop_0.jpg,42602,2741,1723,SIFT,1897,114,False
43,43,100,selfies,0,./results/cropped/e-ktp\100_selfie_crop_0.jpeg,1281,599,374,102,cards,0,./results/cropped/e-ktp\102_card_crop_0.jpg,32888,2129,1380,SIFT,1281,49,False


In [255]:
scenario_2_sift_without_downscale_df.describe()

,Unnamed: 0,query_image_index,query_image_type_index,query_total_keypoints,query_image_width,query_image_height,train_image_index,train_image_type_index,train_total_keypoints,train_image_width,train_image_height,total_matches,total_good_matches
count,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000,17443.000000
mean,69690.229146,79.099811,0.046437,4939.088058,1083.453649,718.099352,66.800092,0.049074,8235.399587,1640.867282,1081.752565,4939.088058,133.271456
std,40094.520209,43.169005,0.226446,9204.571495,873.644643,572.593453,31.496304,0.250214,11246.355376,1035.086803,672.970402,9204.571495,403.658300
min,11.000000,7.000000,0.000000,6.000000,103.000000,92.000000,7.000000,0.000000,26.000000,182.000000,98.000000,6.000000,0.000000
25%,35033.000000,36.000000,0.000000,716.000000,531.000000,363.000000,42.000000,0.000000,1812.000000,731.000000,495.000000,716.000000,26.000000
50%,69851.000000,101.000000,0.000000,1841.000000,739.000000,509.000000,68.000000,0.000000,3083.000000,1233.000000,815.000000,1841.000000,75.000000
75%,104285.000000,117.000000,0.000000,3863.000000,1314.000000,883.000000,89.000000,0.000000,8826.000000,2585.000000,1690.000000,3863.000000,158.000000
max,139315.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,138.000000,2.000000,55958.000000,4471.000000,2980.000000,55958.000000,43499.000000


##### ORB Scenario 2

In [ ]:
scenario_2_orb_df = scenario_2_df[(scenario_2_df.matching_method == "ORB")]

In [ ]:
scenario_2_orb_df.describe()

In [ ]:
scenario_2_orb_df.tail()

In [ ]:
# Downscaled dataframe
scenario_2_orb_with_downscale_df = scenario_2_orb_df[scenario_2_orb_df.with_resize == True]
scenario_2_orb_with_downscale_df.head()

In [ ]:
scenario_2_orb_with_downscale_df.describe()

In [ ]:
# col = scenario_2_orb_df[scenario_2_orb_df.total_good_matches < 100]
col = scenario_2_orb_with_downscale_df
col.count()

In [ ]:
# Without downscaled dataframe
scenario_2_orb_without_downscale_df = scenario_2_orb_df[scenario_2_orb_df.with_resize == False]
scenario_2_orb_without_downscale_df.head()

In [ ]:
scenario_2_orb_without_downscale_df.describe()